In [2]:
import os
import gc
from dataclasses import dataclass
from typing import Optional

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report
)
from sklearn.utils.class_weight import compute_class_weight

from tqdm.notebook import tqdm

In [3]:
RANDOM_SEED = 42
BATCH_SIZE = 16
MAX_LENGTH = 2000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "sergeyzh/rubert-tiny-turbo"

## Data Import

In [4]:
## General Data for first step of transfer learning
data_general = pd.read_csv("/kaggle/input/nexign/datasets.csv")
data_general = data_general.drop(data_general.columns[0], axis=1)
data_general["len"] = data_general["text"].apply(len)
data_general = data_general.loc[data_general["len"]<=MAX_LENGTH].copy()

train_general, temp_general = train_test_split(
    data_general, test_size=0.3, random_state=RANDOM_SEED
)

test_general, val_general = train_test_split(
    temp_general, test_size=0.5, random_state=RANDOM_SEED
)

In [9]:
data_general.shape

(198680, 3)

In [5]:
## Mistral Labeled 500 sentences
mistral_labeled = pd.read_csv("/kaggle/input/nexign-original-data/comments_500_mistral_labeles_clear.csv")

In [6]:
## Mistral Generated synth data
synth_val = pd.read_csv("/kaggle/input/new-nexign-data/test_synthetic (1).csv")
synth_train = pd.read_csv("/kaggle/input/new-nexign-data/train_synthetic (1).csv")

In [19]:
len(synth_train) + len(synth_val)

4778

In [7]:
## Test data labeled by nexign
clear_test = pd.read_csv("/kaggle/input/clear-test/comments_100.csv")
clear_test = clear_test.dropna()

## Datasets prep

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(texts):
    return tokenizer(
        texts.tolist(),
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors="pt"
    )

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }
    
    def __len__(self):
        return len(self.labels)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

In [8]:
train_general_encodings = tokenize(train_general["text"].values)
test_general_encodings = tokenize(test_general["text"].values)
val_general_encodings = tokenize(val_general["text"].values)

train_general_dataset = SentimentDataset(train_general_encodings, train_general["sentiment"].values)
val_general_dataset = SentimentDataset(val_general_encodings, val_general["sentiment"].values)
test_general_dataset = SentimentDataset(test_general_encodings, test_general["sentiment"].values)

train_general_loader = DataLoader(train_general_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_general_loader = DataLoader(val_general_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_general_loader = DataLoader(test_general_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
mistral_labeled_encodings = tokenize(mistral_labeled["MessageText"].values)
synth_val_encodings = tokenize(synth_val["MessageText"].values)
synth_train_encodings = tokenize(synth_train["MessageText"].values)
clear_test_encodings = tokenize(clear_test["MessageText"].values)

mistral_labeled_dataset = SentimentDataset(mistral_labeled_encodings, mistral_labeled["final_label"].values)
synth_val_dataset = SentimentDataset(synth_val_encodings, synth_val["final_label"].values)
synth_train_dataset = SentimentDataset(synth_train_encodings, synth_train["final_label"].values)
clear_test_dataset = SentimentDataset(clear_test_encodings, clear_test["Class"].values)

mistral_labeled_loader = DataLoader(mistral_labeled_dataset, batch_size=BATCH_SIZE, shuffle=True)
synth_val_loader = DataLoader(synth_val_dataset, batch_size=BATCH_SIZE, shuffle=True)
synth_train_loader = DataLoader(synth_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
clear_test_loader = DataLoader(clear_test_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Classes for training and evaluation

In [10]:
@dataclass
class TrainingConfig:
    batch_size: int = 16
    epochs: int = 10
    learning_rate: float = 2e-5
    eps: float = 1e-8
    warmup_ratio: float = 0.1
    model_name: str = "sergeyzh/rubert-tiny-turbo"
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    output_dir: str = "./output"
    grad_clip: float = 1.0

    def create_output_dir(self):
        """Create the output directory if it does not exist."""
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
            print(f"Created output directory at {self.output_dir}")

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }
    
    def __len__(self):
        return len(self.labels)

class ModelTrainer:
    def __init__(self, model, train_loader, val_loader, config=TrainingConfig()):
        self.model = model.to(config.device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.config = config
        self.config.create_output_dir()
        self.optimizer = AdamW(self.model.parameters(), lr=config.learning_rate, eps=config.eps)
        self.scheduler = self._create_scheduler()
        self.best_val_loss = float('inf')
        self.metrics = []
        self.class_weights = self._calculate_class_weights()
        if self.class_weights is not None:
            self.criterion = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(config.device))
        else:
            self.criterion = None
        
    def _calculate_class_weights(self):
        """Calculate class weights from the training dataset."""
        labels = []
        for batch in self.train_loader:
            labels.extend(batch["labels"].tolist())
        class_weights = compute_class_weight("balanced", classes=torch.unique(torch.tensor(labels)).tolist(), y=labels)
        return torch.tensor(class_weights, dtype=torch.float32)
    
    def _create_scheduler(self):
        total_steps = len(self.train_loader) * self.config.epochs
        warmup_steps = int(total_steps * self.config.warmup_ratio)
        return get_linear_schedule_with_warmup(
            self.optimizer, 
            num_warmup_steps=warmup_steps,
            num_training_steps=total_steps
        )
    
    def train_epoch(self, epoch):
        self.model.train()
        epoch_loss = 0
        progress_bar = tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.epochs} [Train]")
        
        for batch in progress_bar:
            self.optimizer.zero_grad()
            inputs = self._prepare_inputs(batch)
            outputs = self.model(**inputs)
            logits = outputs.logits
            labels = inputs["labels"]
            if self.criterion is not None:
                loss = self.criterion(logits, labels)
            else:
                loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.grad_clip)
            self.optimizer.step()
            self.scheduler.step()
            
            epoch_loss += loss.item()
            progress_bar.set_postfix({"loss": loss.item()})
            
        return epoch_loss / len(self.train_loader)
    
    def validate(self):
        self.model.eval()
        epoch_loss = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch in tqdm(self.val_loader, desc="Validating"):
                inputs = self._prepare_inputs(batch)
                outputs = self.model(**inputs)
                logits = outputs.logits
                labels = inputs["labels"]
                if self.criterion is not None:
                    loss = self.criterion(logits, labels)
                else:
                    loss = outputs.loss
                epoch_loss += loss.item()
                
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())
                
        avg_loss = epoch_loss / len(self.val_loader)
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average="weighted")
        
        return avg_loss, accuracy, f1
    
    def _prepare_inputs(self, batch):
        return {
            "input_ids": batch["input_ids"].to(self.config.device),
            "attention_mask": batch["attention_mask"].to(self.config.device),
            "labels": batch["labels"].to(self.config.device)
        }
    
    def train(self):
        for epoch in range(self.config.epochs):
            train_loss = self.train_epoch(epoch)
            val_loss, val_acc, val_f1 = self.validate()
            
            self.metrics.append({
                "epoch": epoch+1,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_accuracy": val_acc,
                "val_f1": val_f1
            })
            
            print(f"\nEpoch {epoch+1}/{self.config.epochs}")
            print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
            print(f"Val Accuracy: {val_acc:.4f} | Val F1: {val_f1:.4f}\n")
            
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.save_model()
                
        self.plot_metrics()
        self.save_metrics_to_excel()
        return self.metrics
    
    def save_model(self):
        torch.save(self.model.state_dict(), f"{self.config.output_dir}/best_model.pt")
    
    def plot_metrics(self):
        try:
            import matplotlib.pyplot as plt
            plt.style.use('ggplot')
        except ImportError:
            print("Matplotlib is required to plot metrics. Please install it.")
            return

        epochs = [m['epoch'] for m in self.metrics]
        train_loss = [m['train_loss'] for m in self.metrics]
        val_loss = [m['val_loss'] for m in self.metrics]
        val_f1 = [m['val_f1'] for m in self.metrics]

        fig, ax1 = plt.subplots(figsize=(10, 6))

        color = 'tab:blue'
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss', color=color)
        ax1.plot(epochs, train_loss, label='Train Loss', color=color, linestyle='-', linewidth=2)
        ax1.plot(epochs, val_loss, label='Val Loss', color='tab:orange', linestyle='-', linewidth=2)
        ax1.tick_params(axis='y', labelcolor=color)
        ax1.legend(loc='upper left')

        ax2 = ax1.twinx()
        color = 'tab:green'
        ax2.set_ylabel('F1 Score', color=color)
        ax2.plot(epochs, val_f1, label='Val F1', color=color, linestyle='--', linewidth=2)
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.legend(loc='upper right')

        plt.title('Training Loss, Validation Loss, and Validation F1 Score')
        fig.tight_layout()
        plt.savefig(os.path.join(self.config.output_dir, 'loss_f1_plot.png'), bbox_inches='tight', dpi=300)
        plt.close()
    
    def save_metrics_to_excel(self):
        """Save training/validation metrics to an Excel file."""
        df = pd.DataFrame(self.metrics)
        df.to_excel(os.path.join(self.config.output_dir, "training_metrics.xlsx"), index=False)
        print(f"Metrics saved to {os.path.join(self.config.output_dir, 'training_metrics.xlsx')}")

class OutputLayerTrainer(ModelTrainer):
    def __init__(self, model, train_loader, val_loader, config=TrainingConfig()):
        super().__init__(model, train_loader, val_loader, config)
        self._freeze_base_model()
        
    def _freeze_base_model(self):
        for param in self.model.base_model.parameters():
            param.requires_grad = False
        for param in self.model.classifier.parameters():
            param.requires_grad = True
        print("Base model frozen. Only the output layer will be trained.")
    
    def train(self):
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print(f"Trainable parameters: {trainable_params}")
        return super().train()

In [11]:
class ModelEvaluator:
    def __init__(self, model, test_loader, config=TrainingConfig()):
        self.model = model.to(config.device)
        self.test_loader = test_loader
        self.config = config
        self.config.create_output_dir()
        self.class_names = ['NEUTRAL', 'POSITIVE', 'NEGATIVE']
        
    def evaluate(self):
        self.model.eval()
        test_loss = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch in tqdm(self.test_loader, desc="Testing"):
                inputs = {
                    "input_ids": batch["input_ids"].to(self.config.device),
                    "attention_mask": batch["attention_mask"].to(self.config.device),
                    "labels": batch["labels"].to(self.config.device)
                }
                outputs = self.model(**inputs)
                loss = outputs.loss
                test_loss += loss.item()
                
                logits = outputs.logits
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(inputs["labels"].cpu().numpy())
                
        avg_loss = test_loss / len(self.test_loader)
        metrics = self._calculate_metrics(avg_loss, all_labels, all_preds)
        self._save_results(metrics, all_labels, all_preds)
        return metrics
    
    def _calculate_metrics(self, avg_loss, true_labels, preds):
        return {
            "test_loss": avg_loss,
            "accuracy": accuracy_score(true_labels, preds),
            "f1_weighted": f1_score(true_labels, preds, average="weighted"),
            "precision_macro": precision_score(true_labels, preds, average='macro'),
            "recall_macro": recall_score(true_labels, preds, average='macro'),
            "classification_report": classification_report(true_labels, preds, target_names=self.class_names)
        }
    
    def _save_results(self, metrics, true_labels, preds):
        pd.DataFrame([metrics]).to_csv(f"{self.config.output_dir}/test_metrics.csv", index=False)
        self._plot_confusion_matrix(true_labels, preds)
        
    def _plot_confusion_matrix(self, true_labels, preds):
        cm = confusion_matrix(true_labels, preds)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=self.class_names, 
                    yticklabels=self.class_names)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.savefig(f"{self.config.output_dir}/confusion_matrix.png", bbox_inches='tight')
        plt.close()

class ONNXExporter:
    def __init__(self, model, tokenizer, config=TrainingConfig()):
        self.model = model
        self.tokenizer = tokenizer
        self.config = config
        
    def export(self, dummy_text="Пример текста"):
        class WrappedModel(torch.nn.Module):
            def __init__(self, model):
                super().__init__()
                self.model = model

            def forward(self, input_ids, attention_mask):
                return self.model(input_ids, attention_mask).logits
                
        wrapped_model = WrappedModel(self.model).eval()
        inputs = self.tokenizer(dummy_text, return_tensors="pt")
        
        torch.onnx.export(
            wrapped_model,
            (inputs["input_ids"], inputs["attention_mask"]),
            f"{self.config.output_dir}/model.onnx",
            input_names=['input_ids', 'attention_mask'],
            output_names=['logits'],
            dynamic_axes={
                'input_ids': {0: 'batch_size', 1: 'sequence_length'},
                'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
                'logits': {0: 'batch_size'}
            },
            opset_version=14
        )


## Training base model

In [12]:
config_general = TrainingConfig(output_dir="./general_model/")
model = AutoModelForSequenceClassification.from_pretrained(config_general.model_name, num_labels=3)

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sergeyzh/rubert-tiny-turbo and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
#trainer = ModelTrainer(model, train_general_loader, val_general_loader, config_general)
#metrics = trainer.train()

In [14]:
model.load_state_dict(torch.load(f"/kaggle/input/best_model_nxign/pytorch/default/1/best_model.pt"))
evaluator = ModelEvaluator(model, test_general_loader, config_general)
test_metrics = evaluator.evaluate()

<ipython-input-14-b520ed4323d3>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"/kaggle/input/best_model_nxign/pytorch/default/1/best_mod

Created output directory at ./general_model/


Testing: 100%|██████████| 1554/1554 [01:07<00:00, 22.92it/s]


In [15]:
def print_test_metrics(metrics):
    for i in metrics.keys():
        if i!="classification_report":
            print(f"{i}: {metrics[i]}")
        else:
            print(f"{i}")
            print(f"{metrics[i]}")

print_test_metrics(test_metrics)

test_loss: 0.4268440281926742
accuracy: 0.8149668074834038
f1_weighted: 0.8135819989080717
precision_macro: 0.8003812232810521
recall_macro: 0.7999252540211469
classification_report
              precision    recall  f1-score   support

     NEUTRAL       0.72      0.67      0.70      6064
    POSITIVE       0.85      0.85      0.85     12227
    NEGATIVE       0.83      0.87      0.85      6564

    accuracy                           0.81     24855
   macro avg       0.80      0.80      0.80     24855
weighted avg       0.81      0.81      0.81     24855



In [16]:
config_general = TrainingConfig(output_dir="./general_model_on_clear_data/")
evaluator = ModelEvaluator(model, clear_test_loader, config_general)
test_metrics_clear = evaluator.evaluate()

print_test_metrics(test_metrics_clear)

Created output directory at ./general_model_on_clear_data/


Testing: 100%|██████████| 7/7 [00:00<00:00, 25.69it/s]


test_loss: 1.0221395918301173
accuracy: 0.5858585858585859
f1_weighted: 0.6084933394897443
precision_macro: 0.5280492898913952
recall_macro: 0.5507271002364669
classification_report
              precision    recall  f1-score   support

     NEUTRAL       0.42      0.76      0.54        21
    POSITIVE       0.93      0.63      0.75        59
    NEGATIVE       0.24      0.26      0.25        19

    accuracy                           0.59        99
   macro avg       0.53      0.55      0.51        99
weighted avg       0.69      0.59      0.61        99



In [17]:
config_synth= TrainingConfig(output_dir="./synth_model/")
model.load_state_dict(torch.load(f"/kaggle/input/best_model_nxign/pytorch/default/1/best_model.pt"))
trainer = ModelTrainer(model, synth_train_loader, synth_val_loader, config_synth)
metrics = trainer.train()

<ipython-input-17-0bb371362b3b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"/kaggle/input/best_model_nxign/pytorch/default/1/best_mod

Created output directory at ./synth_model/


Validating: 100%|██████████| 90/90 [00:03<00:00, 23.28it/s]



Epoch 1/10
Train Loss: 1.2336 | Val Loss: 0.8258
Val Accuracy: 0.6491 | Val F1: 0.6547



Validating: 100%|██████████| 90/90 [00:03<00:00, 23.30it/s]


...


Epoch 8/10
Train Loss: 0.1900 | Val Loss: 1.2925
Val Accuracy: 0.7053 | Val F1: 0.7030



Validating: 100%|██████████| 90/90 [00:03<00:00, 23.29it/s]



Epoch 9/10
Train Loss: 0.1766 | Val Loss: 1.3546
Val Accuracy: 0.7067 | Val F1: 0.7042



Validating: 100%|██████████| 90/90 [00:03<00:00, 23.24it/s]



Epoch 10/10
Train Loss: 0.1504 | Val Loss: 1.3374
Val Accuracy: 0.7060 | Val F1: 0.7036

Metrics saved to ./synth_model/training_metrics.xlsx


In [18]:
model.load_state_dict(torch.load(f"/kaggle/working/synth_model/best_model.pt"))
evaluator = ModelEvaluator(model, mistral_labeled_loader, config_synth)
test_metrics_synth = evaluator.evaluate()

<ipython-input-18-cd9baa6e71ab>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"/kaggle/working/synth_model/best_model.pt"))
Testing: 100

In [19]:
print_test_metrics(test_metrics_synth)

test_loss: 0.6477523595094681
accuracy: 0.7267326732673267
f1_weighted: 0.7330365598786713
precision_macro: 0.6925221772223368
recall_macro: 0.7153144870910902
classification_report
              precision    recall  f1-score   support

     NEUTRAL       0.74      0.72      0.73       229
    POSITIVE       0.85      0.75      0.80       200
    NEGATIVE       0.49      0.67      0.56        76

    accuracy                           0.73       505
   macro avg       0.69      0.72      0.70       505
weighted avg       0.75      0.73      0.73       505



In [20]:
config_synth.output_dir = "./synth_model_on_clear"
model.load_state_dict(torch.load(f"/kaggle/working/synth_model/best_model.pt"))
evaluator = ModelEvaluator(model, clear_test_loader, config_synth)
test_metrics_synth_on_clear = evaluator.evaluate()

<ipython-input-20-902bd5acc465>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"/kaggle/working/synth_model/best_model.pt"))


Created output directory at ./synth_model_on_clear


Testing: 100%|██████████| 7/7 [00:00<00:00, 25.80it/s]


In [21]:
print_test_metrics(test_metrics_synth_on_clear)

test_loss: 0.3688892956290926
accuracy: 0.8585858585858586
f1_weighted: 0.8531090960758482
precision_macro: 0.8206576850644648
recall_macro: 0.8067768290783456
classification_report
              precision    recall  f1-score   support

     NEUTRAL       0.85      0.52      0.65        21
    POSITIVE       0.95      0.95      0.95        59
    NEGATIVE       0.67      0.95      0.78        19

    accuracy                           0.86        99
   macro avg       0.82      0.81      0.79        99
weighted avg       0.87      0.86      0.85        99



## Tuning classificator head

In [22]:
model.load_state_dict(torch.load(f"/kaggle/input/best_model_nxign/pytorch/default/1/best_model.pt"))
config_output_general = TrainingConfig(output_dir = "./general_output_tuning",
                                        epochs=100,
                                        learning_rate=1e-4,
                                        warmup_ratio=0.0,
                                        grad_clip=1.0   
                                      )

output_layer_trainer = OutputLayerTrainer(model, mistral_labeled_loader, clear_test_loader, config_output_general)
metrics = output_layer_trainer.train()

<ipython-input-22-87f5c1297473>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"/kaggle/input/best_model_nxign/pytorch/default/1/best_mod

Created output directory at ./general_output_tuning
Base model frozen. Only the output layer will be trained.
Trainable parameters: 939


Validating: 100%|██████████| 7/7 [00:00<00:00, 25.91it/s]



Epoch 1/100
Train Loss: 1.5019 | Val Loss: 1.0385
Val Accuracy: 0.6162 | Val F1: 0.6381



Validating: 100%|██████████| 7/7 [00:00<00:00, 25.91it/s]


...


Epoch 98/100
Train Loss: 0.7868 | Val Loss: 0.5434
Val Accuracy: 0.8182 | Val F1: 0.8110



Validating: 100%|██████████| 7/7 [00:00<00:00, 25.72it/s]



Epoch 99/100
Train Loss: 0.8032 | Val Loss: 0.5091
Val Accuracy: 0.8182 | Val F1: 0.8110



Validating: 100%|██████████| 7/7 [00:00<00:00, 25.91it/s]



Epoch 100/100
Train Loss: 0.8102 | Val Loss: 0.5604
Val Accuracy: 0.8182 | Val F1: 0.8110

Metrics saved to ./general_output_tuning/training_metrics.xlsx


In [23]:
model.load_state_dict(torch.load(f"/kaggle/working/general_output_tuning/best_model.pt"))
evaluator = ModelEvaluator(model, clear_test_loader, config_output_general)
test_metrics_output_general_on_clear = evaluator.evaluate()

<ipython-input-23-943bc47b5c46>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"/kaggle/working/general_output_tuning/best_model.pt"))
Te

In [24]:
print_test_metrics(test_metrics_output_general_on_clear)

test_loss: 0.4912905182157244
accuracy: 0.8181818181818182
f1_weighted: 0.8110247126640567
precision_macro: 0.7587962962962962
recall_macro: 0.7366013904818544
classification_report
              precision    recall  f1-score   support

     NEUTRAL       0.69      0.52      0.59        21
    POSITIVE       0.89      0.95      0.92        59
    NEGATIVE       0.70      0.74      0.72        19

    accuracy                           0.82        99
   macro avg       0.76      0.74      0.74        99
weighted avg       0.81      0.82      0.81        99



In [25]:
model.load_state_dict(torch.load(f"/kaggle/working/synth_model/best_model.pt"))
config_output_general.output_dir = "./synth_output_tuning"
output_layer_trainer = OutputLayerTrainer(model, mistral_labeled_loader, clear_test_loader, config_output_general)
metrics = output_layer_trainer.train()

<ipython-input-25-9b80a051612f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"/kaggle/working/synth_model/best_model.pt"))
/usr/local/l

Created output directory at ./synth_output_tuning
Base model frozen. Only the output layer will be trained.
Trainable parameters: 939


Validating: 100%|██████████| 7/7 [00:00<00:00, 25.64it/s]



Epoch 1/100
Train Loss: 0.6990 | Val Loss: 0.3539
Val Accuracy: 0.8586 | Val F1: 0.8531



Validating: 100%|██████████| 7/7 [00:00<00:00, 25.76it/s]


...


Epoch 98/100
Train Loss: 0.6283 | Val Loss: 0.3609
Val Accuracy: 0.8687 | Val F1: 0.8646



Validating: 100%|██████████| 7/7 [00:00<00:00, 25.66it/s]



Epoch 99/100
Train Loss: 0.6576 | Val Loss: 0.3666
Val Accuracy: 0.8687 | Val F1: 0.8646



Validating: 100%|██████████| 7/7 [00:00<00:00, 25.81it/s]



Epoch 100/100
Train Loss: 0.6380 | Val Loss: 0.4699
Val Accuracy: 0.8687 | Val F1: 0.8646

Metrics saved to ./synth_output_tuning/training_metrics.xlsx


In [26]:
model.load_state_dict(torch.load(f"/kaggle/working/synth_output_tuning/best_model.pt"))
evaluator = ModelEvaluator(model, clear_test_loader, config_output_general)
test_metrics_output_synth_on_clear = evaluator.evaluate()

<ipython-input-26-ef2d24fa06eb>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"/kaggle/working/synth_output_tuning/best_model.pt"))
Test

In [27]:
print_test_metrics(test_metrics_output_synth_on_clear)

test_loss: 0.34385326504707336
accuracy: 0.8686868686868687
f1_weighted: 0.8646464646464648
precision_macro: 0.832867697274477
recall_macro: 0.8226498449513614
classification_report
              precision    recall  f1-score   support

     NEUTRAL       0.86      0.57      0.69        21
    POSITIVE       0.95      0.95      0.95        59
    NEGATIVE       0.69      0.95      0.80        19

    accuracy                           0.87        99
   macro avg       0.83      0.82      0.81        99
weighted avg       0.88      0.87      0.86        99



In [33]:
onnx_export = ONNXExporter(model.to("cpu"), tokenizer, config_output_general)

In [34]:
onnx_export.export()